In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
import pathlib

In [4]:
import xarray as xr

### list of variables to update 

- MSLP 
- UWND 
- VWND
- OMEGA

### some parameters 

In [67]:
time_slice = {"time":slice("1948","2021")}

### first check the current datasets format and metadata using one dataset 

In [72]:
dset = xr.open_dataset('/media/nicolasf/END19101/data/PICT/datasets/NCEP/NCEP1_monthly_wind_1948_2014.nc') 

In [73]:
dset

<xarray.Dataset>
Dimensions:     (time: 804, latitudes: 73, longitudes: 144)
Coordinates:
  * longitudes  (longitudes) float32 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * time        (time) datetime64[ns] 1948-01-01 1948-02-01 ... 2014-12-01
  * latitudes   (latitudes) float32 90.0 87.5 85.0 82.5 ... -85.0 -87.5 -90.0
Data variables:
    vwnd_1000   (time, latitudes, longitudes) float64 ...
    vwnd_200    (time, latitudes, longitudes) float64 ...
    uwnd_850    (time, latitudes, longitudes) float64 ...
    uwnd_1000   (time, latitudes, longitudes) float64 ...
    vwnd_850    (time, latitudes, longitudes) float64 ...
    uwnd_200    (time, latitudes, longitudes) float64 ...

### list of URLs from the PSL 

- MSLP: https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.derived/surface/slp.mon.mean.nc.html 
  
 - u-wind: https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/uwnd.mon.mean.nc.html 
  
- vector wind: https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/uwnd.mon.mean.nc.html and https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/vwnd.mon.mean.nc.html 

in one dataset (NCEP1_monthly_wind_1948_2021.nc), levels 1000, 850, 200 

- HGT: https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/hgt.mon.mean.nc.html 

    levels: 1000, 850, 200 

- Omega: https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/omega.mon.mean.nc.html

    levels: 500 (variable name: 'omega_500')

### MSLP 

In [21]:
mslp = xr.open_dataset("https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.derived/surface/slp.mon.mean.nc")

In [22]:
dict_rename = {'lat':'latitudes', 'lon':'longitudes'}

In [23]:
mslp = mslp.rename(dict_rename)

In [24]:
mslp = mslp.sel(time_slice) 

In [25]:
mslp.to_netcdf("/media/nicolasf/END19101/data/PICT/datasets/NCEP/NCEP1_monthly_mslp_1948_2021.nc") 

### HGT 

In [26]:
hgt = xr.open_dataset("https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/hgt.mon.mean.nc")

In [27]:
dict_rename = {'lat':'latitudes', 'lon':'longitudes'}

In [28]:
hgt = hgt.rename(dict_rename)

In [29]:
hgt = hgt.sel(time_slice) 

In [41]:
outfile = hgt[['time','latitudes','longitudes']]

In [42]:
for level in [1000, 850, 200]: 
    
    outfile[f"hgt_{level}"] = hgt.sel(level=level)['hgt']

In [45]:
outfile.to_netcdf("/media/nicolasf/END19101/data/PICT/datasets/NCEP/NCEP1_monthly_hgt_1948_2021.nc")

### Omega 

In [46]:
omega = xr.open_dataset("https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/omega.mon.mean.nc")

In [50]:
omega = omega.rename(dict_rename)

In [51]:
omega = omega.sel(time_slice) 

In [55]:
outfile = omega[['time','latitudes','longitudes']]

In [56]:
outfile['omega_500'] = omega.sel(level=500)['omega']

In [57]:
outfile.to_netcdf("/media/nicolasf/END19101/data/PICT/datasets/NCEP/NCEP1_monthly_omega_1948_2021.nc")

### Vector winds 

## take from uwnd and vwnd, then merge 

In [62]:
uwnd = xr.open_dataset("https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/uwnd.mon.mean.nc")

In [63]:
vwnd = xr.open_dataset("https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/vwnd.mon.mean.nc")

In [64]:
uwnd = uwnd.rename(dict_rename)

In [65]:
vwnd = vwnd.rename(dict_rename)

In [68]:
uwnd = uwnd.sel(time_slice)

In [69]:
vwnd = vwnd.sel(time_slice)

### template for wind outfile 

In [71]:
outfile = uwnd[['time','latitudes','longitudes']]

In [74]:
for level in [1000, 850, 200]: 
    
    outfile[f"uwnd_{level}"] = uwnd.sel(level=level)['uwnd']
    outfile[f"vwnd_{level}"] = vwnd.sel(level=level)['vwnd']